In [8]:
from tqdm import tqdm
import numpy as np
import MDAnalysis as mda

import pandas as pd
from pathlib import Path
import MDAnalysis.transformations as trans

import matplotlib.pyplot as plt
from dotenv import load_dotenv
load_dotenv()

from utils.classes import Peptide

In [9]:
# Load simulations
step_size = 500 
membrane_lipids = 512
peptide_name = "WF2_8_1"

peptide = Peptide(
    pep_name=peptide_name,
    xtc_file_path="/Volumes/miru_back/young_concentration_WF2/WF2_8/md_0_1_combined_first500ns_pbc.xtc",
    tpr_file_path="/Volumes/miru_back/young_concentration_WF2/WF2_8/md_0_1.tpr",
    peptide_number=8, amino_acid_count=25, step_size=step_size)

In [10]:
def get_membrane_c2(obj):

    start_sim, stop_sim, step = obj.u.trajectory.check_slice_indices(None, None, 50)
    frames = np.arange(start_sim, stop_sim, 50)
    n_frames = frames.size
    water_time = []

    for frame_index, _ in tqdm(enumerate(obj.u.trajectory[frames]), total=n_frames):
        p_memb = obj.u.select_atoms(f'resname POPG and name C2').positions[:,[2]].astype(float)
        c2_up = np.min(p_memb[:membrane_lipids/2])
        c2_low = np.max(p_memb[membrane_lipids/2:])
        mid_c2 =  (c2_up + c2_low)/2
        water_mols = obj.u.select_atoms(f"(resname SOL and name OW) and (prop abs z < {c2_up}) and (prop abs z > {mid_c2})")
        water_time.append([frame_index, len(water_mols), c2_up, mid_c2])
        
    return water_time

In [11]:

def calc_and_write_to_file(obj):
    obj_name = obj.pep_name
    results_directory = f"memb_hydr"
    Path(results_directory).mkdir(parents=True, exist_ok=True)
    data = get_membrane_c2(obj)
    df = pd.DataFrame(data, columns=["Time frame", "Waters", "C_up", "c_mid"])
    df.to_csv(f"memb_hydr/memb_hydr_{obj_name}.csv")
    print(f"{obj_name} --- DONE")
    return data

In [7]:
calc_and_write_to_file(peptide)

100%|██████████| 501/501 [01:22<00:00,  6.04it/s]

WF2_8_1 --- DONE


[[0, 0, 94.57000732421875, 94.5150032043457],
 [1, 0, 95.70999908447266, 96.03500366210938],
 [2, 1, 99.45000457763672, 98.79000473022461],
 [3, 4, 101.47000122070312, 97.81500244140625],
 [4, 2, 101.75, 98.84000396728516],
 [5, 0, 103.87000274658203, 98.08000183105469],
 [6, 3, 101.83999633789062, 96.16999816894531],
 [7, 1, 102.52000427246094, 97.26000595092773],
 [8, 0, 104.54000854492188, 99.21500778198242],
 [9, 1, 105.65000915527344, 100.39000701904297],
 [10, 1, 100.73001098632812, 96.01500701904297],
 [11, 0, 102.89000701904297, 97.89000701904297],
 [12, 4, 102.96000671386719, 97.31500625610352],
 [13, 0, 103.5300064086914, 97.29000473022461],
 [14, 1, 104.33000183105469, 97.58000183105469],
 [15, 0, 102.6300048828125, 95.7350082397461],
 [16, 0, 104.07000732421875, 97.99000549316406],
 [17, 2, 105.1500015258789, 98.10500717163086],
 [18, 9, 106.61000061035156, 97.89500045776367],
 [19, 3, 105.3800048828125, 98.09500503540039],
 [20, 5, 104.49000549316406, 96.13500595092773],
 